# This Notebook takes a video and creates bounding boxes and segmentations of fibers for each frame.

## Import Required Libraries

In [8]:
# Imports
import detectron2
import contextlib
import datetime
import io
import os
import json
import logging
import cv2
import random
import numpy as np
import copy,torch,torchvision
import PIL
from PIL import Image
import xml.etree.ElementTree as X
import math
from itertools import repeat
import itertools
import re
import shutil
import io
import ast

from fvcore.common.file_io import PathManager
from fvcore.common.timer import Timer

from detectron2.structures import Boxes, BoxMode, PolygonMasks
from detectron2.config import *
from detectron2.modeling import build_model
from detectron2 import model_zoo
from detectron2.data import transforms as T
from detectron2.data import detection_utils as utils
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import RotatedCOCOEvaluator,DatasetEvaluators, inference_on_dataset, coco_evaluation,DatasetEvaluator
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

import matplotlib.pyplot as plt
from platform import python_version

import glob
import time
import shutil
from multiprocessing.pool import ThreadPool
import concurrent.futures
from pathlib import Path
import shutil

#from pyunpack import Archive

import torch
torch.cuda.set_device(0)

from torch.utils.cpp_extension import CUDA_HOME
print(torch.cuda.is_available(), CUDA_HOME)

print('\n')

setup_logger()

print('Python version:',python_version(),'\nOpencv version:',cv2.__version__,'\nTorch version:', torch.__version__, '\nTorch - Cuda is available:', torch.cuda.is_available(),'\nCudnn is enabled:', torch.backends.cudnn.enabled, '\nCuda version:', torch.version.cuda,'\nTorch_vision version:', torchvision.__version__)


"""
PointRend Training Script.
This script is a simplified version of the training script in detectron2/tools.
"""

import os

import detectron2.data.transforms as T
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, MetadataCatalog, build_detection_train_loader
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    SemSegEvaluator,
    verify_results,
)
from detectron2.projects.point_rend import ColorAugSSDTransform, add_pointrend_config
from tqdm import tqdm
from PIL import Image
import os 
from datetime import datetime
import pandas as pd

input_dir = r"C:\Users\AVLguest\Fiber Semantic Segmentation\test"
combined_results_path = r'C:\Users\AVLguest\Fiber Semantic Segmentation\Segmentation_output'

True C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3


Python version: 3.9.12 
Opencv version: 4.7.0 
Torch version: 1.10.1 
Torch - Cuda is available: True 
Cudnn is enabled: True 
Cuda version: 11.3 
Torch_vision version: 0.11.2


## Setup of Faster RCNN Model

Models are already trained and the saved models can be accessed in the models folder.

In [9]:
"""
Rotated Faster RCNN configs
"""

cfg_RFRCNN = get_cfg()

# Change the path here to your desired path and place the given model in that folder.
cfg_RFRCNN.OUTPUT_DIR = r'C:\Users\AVLguest\Fiber Semantic Segmentation\model_Faster_RCNN'

#cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

cfg_RFRCNN.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg_RFRCNN.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml") # Let training initialize from model zoo
cfg_RFRCNN.DATASETS.TRAIN = ()
cfg_RFRCNN.DATASETS.TEST = ()

cfg_RFRCNN.MODEL.MASK_ON=False
cfg_RFRCNN.MODEL.PROPOSAL_GENERATOR.NAME = "RRPN"
cfg_RFRCNN.MODEL.RPN.HEAD_NAME = "StandardRPNHead"
cfg_RFRCNN.MODEL.RPN.BBOX_REG_WEIGHTS = (1.0,1.0,1.0,1.0,1.0)
cfg_RFRCNN.MODEL.ANCHOR_GENERATOR.NAME = "RotatedAnchorGenerator"
cfg_RFRCNN.MODEL.ANCHOR_GENERATOR.ANGLES = [[-90,-60,-30,0,30,60,90]]
cfg_RFRCNN.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9 
cfg_RFRCNN.MODEL.ROI_HEADS.NAME = "RROIHeads"
cfg_RFRCNN.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   #this is far lower than usual.  
cfg_RFRCNN.MODEL.ROI_HEADS.NUM_CLASSES =1
cfg_RFRCNN.MODEL.ROI_BOX_HEAD.POOLER_TYPE = "ROIAlignRotated"
cfg_RFRCNN.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS = (1.0,1.0,1.0,1.0,1.0)
# cfg.MODEL.ROI_BOX_HEAD.NUM_CONV=4
# cfg.MODEL.ROI_MASK_HEAD.NUM_CONV=8
cfg_RFRCNN.SOLVER.IMS_PER_BATCH = 14 #can be up to  24 for a p100 (6 default)
cfg_RFRCNN.SOLVER.CHECKPOINT_PERIOD=1000
cfg_RFRCNN.SOLVER.BASE_LR = 0.005
# cfg.SOLVER.GAMMA=0.5
# cfg.SOLVER.STEPS=[1000,2000,4000,8000, 12000]
cfg_RFRCNN.SOLVER.MAX_ITER=5000


cfg_RFRCNN.DATALOADER.NUM_WORKERS = 1
cfg_RFRCNN.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True 
# cfg.DATALOADER.SAMPLER_TRAIN= "RepeatFactorTrainingSampler"
# cfg.DATALOADER.REPEAT_THRESHOLD=0.01
os.makedirs(cfg_RFRCNN.OUTPUT_DIR, exist_ok=True)#lets just check our output dir exists
# cfg.MODEL.BACKBONE.FREEZE_AT=6

In [10]:
"""
Rotated Faster RCNN Predictor
"""
cfg_RFRCNN.MODEL.WEIGHTS = os.path.join(cfg_RFRCNN.OUTPUT_DIR, "model_final.pth")

# predictor = RotatedPredictor(cfg)
predictor_RFRCNN = DefaultPredictor(cfg_RFRCNN)

[08/12 23:11:39 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from C:\Users\AVLguest\Fiber Semantic Segmentation\model_Faster_RCNN\model_final.pth ...
[08/12 23:11:39 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from C:\Users\AVLguest\Fiber Semantic Segmentation\model_Faster_RCNN\model_final.pth ...
[08/12 23:11:39 fvcore.common.checkpoint]: [Checkpointer] Loading from c:\Users\AVLguest\Fiber Semantic Segmentation\model_Faster_RCNN\model_final.pth ...


In [11]:
"""
Pointrend Semantic Segmentation configs
"""

def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_pointrend_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    #cfg.freeze()
    default_setup(cfg, args)
    return cfg

# Additional arguments for cfg
import argparse

args = argparse.Namespace()
args.config_file = r"C:\Users\AVLguest\detectron2\projects\PointRend\configs\SemanticSegmentation\pointrend_semantic_R_101_FPN_1x_cityscapes.yaml "
args.opts = []
args.eval_only = "False"

cfg_PRSS = setup(args)

cfg_PRSS.DATASETS.TRAIN = ("CFH_test_padded_v2_padded",)
cfg_PRSS.DATASETS.TEST = () # () --> if no validation data,
cfg_PRSS.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 2
cfg_PRSS.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg_PRSS.MODEL.POINT_HEAD.NUM_CLASSES  = 2
cfg_PRSS.MODEL.ROI_BOX_HEAD.FED_LOSS_NUM_CLASSES = 2
cfg_PRSS.MODEL.RETINANET.NUM_CLASSES = 2

cfg_PRSS.MODEL.PANOPTIC_FPN.STUFF_AREA_LIMIT = 2896804
cfg_PRSS.MODEL.PANOPTIC_FPN.COMBINE.STUFF_AREA_LIMIT = 2896804

#cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE = 0 # use this if want to ignore fiber class pixels?
cfg_PRSS.SOLVER.IMS_PER_BATCH = 2
cfg_PRSS.SOLVER.STEPS = (16000,20000)
cfg_PRSS.SOLVER.MAX_ITER = 24000
cfg_PRSS.SOLVER.CHECKPOINT_PERIOD = 2000
cfg_PRSS.OUTPUT_DIR = r'C:\Users\AVLguest\Fiber Semantic Segmentation\model_Pointrend'
cfg_PRSS.DATALOADER.NUM_WORKERS = 1 
cfg_PRSS.SOLVER.BASE_LR = 0.00025
cfg_PRSS.INPUT.MASK_FORMAT = "bitmask"
cfg_PRSS.MODEL.MASK_ON = True

# cfg.MODEL.SEM_SEG_HEAD.NORM = "BN" # A hacky way to disable syncbn to avoid errors in training due to request for multi gpu
# cfg.MODEL.RESNETS.NORM = "BN"
#cfg.MODEL.SEM_SEG_HEAD.PROJECT_CHANNELS = [64]
cfg_PRSS.INPUT.CROP.ENABLED = False
os.makedirs(cfg_PRSS.OUTPUT_DIR, exist_ok=True)

[08/12 23:11:39 detectron2]: Rank of current process: 0. World size: 1
[08/12 23:11:39 detectron2]: Rank of current process: 0. World size: 1
[08/12 23:11:40 detectron2]: Environment info:
----------------------  ---------------------------------------------------------------------------------------------
sys.platform            win32
Python                  3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
numpy                   1.23.5
detectron2              0.6 @c:\users\avlguest\detectron2\detectron2
Compiler                MSVC 192930147
CUDA compiler           CUDA 11.3
detectron2 arch flags   c:\users\avlguest\detectron2\detectron2\_C.cp39-win_amd64.pyd; cannot find cuobjdump
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.1 @C:\Users\AVLguest\.conda\envs\CFH\lib\site-packages\torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   NVIDIA TITAN RTX (arch=7.5)
Driver version          531.79
CUDA_HOME               C:\

[08/12 23:11:40 detectron2]: Contents of args.config_file=C:\Users\AVLguest\detectron2\projects\PointRend\configs\SemanticSegmentation\pointrend_semantic_R_101_FPN_1x_cityscapes.yaml :
_BASE_: Base-PointRend-Semantic-FPN.yaml
MODEL:
  WEIGHTS: detectron2://ImageNetPretrained/MSRA/R-101.pkl
  RESNETS:
    DEPTH: 101
  SEM_SEG_HEAD:
    NUM_CLASSES: 19
  POINT_HEAD:
    NUM_CLASSES: 19
    TRAIN_NUM_POINTS: 2048
    SUBDIVISION_NUM_POINTS: 8192
DATASETS:
  TRAIN: ("cityscapes_fine_sem_seg_train",)
  TEST: ("cityscapes_fine_sem_seg_val",)
SOLVER:
  BASE_LR: 0.01
  STEPS: (40000, 55000)
  MAX_ITER: 65000
  IMS_PER_BATCH: 32
INPUT:
  MIN_SIZE_TRAIN: (512, 768, 1024, 1280, 1536, 1792, 2048)
  MIN_SIZE_TRAIN_SAMPLING: "choice"
  MIN_SIZE_TEST: 1024
  MAX_SIZE_TRAIN: 4096
  MAX_SIZE_TEST: 2048
  CROP:
    ENABLED: True
    TYPE: "absolute"
    SIZE: (512, 1024)
    SINGLE_CATEGORY_MAX_AREA: 0.75
  COLOR_AUG_SSD: True
DATALOADER:
  NUM_WORKERS: 10

[08/12 23:11:40 detectron2]: Running with full

[08/12 23:11:40 detectron2]: Running with full config:
CUDNN_BENCHMARK: false
DATALOADER:
  ASPECT_RATIO_GROUPING: true
  FILTER_EMPTY_ANNOTATIONS: true
  NUM_WORKERS: 10
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: []
  PROPOSAL_FILES_TRAIN: []
  TEST:
  - cityscapes_fine_sem_seg_val
  TRAIN:
  - cityscapes_fine_sem_seg_train
GLOBAL:
  HACK: 1.0
INPUT:
  COLOR_AUG_SSD: true
  CROP:
    ENABLED: true
    SINGLE_CATEGORY_MAX_AREA: 0.75
    SIZE:
    - 512
    - 1024
    TYPE: absolute
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 2048
  MAX_SIZE_TRAIN: 4096
  MIN_SIZE_TEST: 1024
  MIN_SIZE_TRAIN:
  - 512
  - 768
  - 1024
  - 1280
  - 1536
  - 1792
  - 2048
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES:
    - - -90
      - 0
      - 90
    ASPECT_RATIOS:
    - - 0.5
      - 1.0
      - 2.0
    NAME: Default

[08/12 23:11:40 detectron2]: Full config saved to ./output\config.yaml
[08/12 23:11:40 detectron2]: Full config saved to ./output\config.yaml
[08/12 23:11:40 d2.utils.env]: Using a generated random seed 40151507
[08/12 23:11:40 d2.utils.env]: Using a generated random seed 40151507


In [12]:
"""
Pointrend Semantic Segmentation Predictor
"""
# In the same folder as above, save the given pointrend model
cfg_PRSS.MODEL.WEIGHTS = os.path.join(cfg_PRSS.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.98   # set a custom testing threshold
predictor_PRSS = DefaultPredictor(cfg_PRSS)

[08/12 23:11:40 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from C:\Users\AVLguest\Fiber Semantic Segmentation\model_Pointrend\model_final.pth ...
[08/12 23:11:40 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from C:\Users\AVLguest\Fiber Semantic Segmentation\model_Pointrend\model_final.pth ...
[08/12 23:11:40 fvcore.common.checkpoint]: [Checkpointer] Loading from c:\Users\AVLguest\Fiber Semantic Segmentation\model_Pointrend\model_final.pth ...


### This routine creates a new flder each and everytime it is called.

In [13]:
def create_new_folder(folder_path): 
    # Get the current date and time 
    now = datetime.now() 
    # Format the date and time as a string 
    folder_name = now.strftime("%Y-%m-%d_%H-%M-%S")
    os.mkdir(os.path.join(folder_path,folder_name))
    return os.path.join(folder_path,folder_name)
#     print("Created new folder:", folder_name) 


### This routine finds the last created sub folder in the given folder

In [17]:
def find_latest_folder(folder_path):
    # Get a list of all folders in the directory
    folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    # Sort the folder names in descending order

    folders.sort(reverse=True)
    if folders:
        latest_folder = folders[0]
#         print("Latest folder:", latest_folder)
        return os.path.join(folder_path, latest_folder)
    else:
        print("No folders found in the directory.")

In [25]:
def convert_and_copy_images(source_folder, destination_folder):
    source_path = Path(source_folder)
    destination_path = Path(destination_folder)

    destination_path.mkdir(parents=True, exist_ok=True)

    image_extensions = ['.jpg', '.jpeg', '.png', '.gif']  # Add more extensions if needed

    image_files = [file for file in source_path.glob('*') if file.suffix.lower() in image_extensions]

    for image_file in image_files:
        img = Image.open(image_file)
        destination_file = destination_path / (image_file.stem + '.png')
        img.save(destination_file)

In [ ]:
def extract_frames(video_path, filename, interval, output_dir):
        vidcap = cv2.VideoCapture(video_path)
        success, image = vidcap.read()
        count = 0
        counter = 0
        if not success:
            return "video is corrupt or unavailable"
        while success:
            if count % interval == 0:
                file_path = os.path.join(output_dir, filename[:-4] + "-" + "frame" + str(count) + ".png")
                if os.path.exists(file_path) == False:
                    cv2.imwrite(file_path, image)
                counter+=1
            success, image = vidcap.read()
            count += 1
        return "Succesfully Extracted all images from the videos"
    
def extract_helper(input_dir, output_dir, interval):
        filenames = [name for name in os.listdir(input_dir) if '.avi' in name]
        for i in tqdm(range(len(filenames))):
            file_path = os.path.join(input_dir, filenames[i])
            msg = extract_frames(file_path, filenames[i], interval, output_dir)
#             print(count, filenames[i])
        return msg

In [26]:
output_dir = create_new_folder(r"C:\Users\AVLguest\Fiber Semantic Segmentation\input_images")
if images:
    convert_and_copy_images(input_dir, output_dir)
else:
    extract_helper(input_dir, output_dir, interval = 1)

In [27]:
# run this this cell for padded images, without this the segmentation will not be accurate 
test_image_path = find_latest_folder(r'C:\Users\AVLguest\Fiber Semantic Segmentation\input_images')
test_images = [name for name in os.listdir(test_image_path) if '.png' in name]

new_path = create_new_folder(r"C:\Users\AVLguest\Fiber Semantic Segmentation\input_padded_images/")
# Load the original image
# Create a new blank image with the desired size and gray background
new_size = (1702, 1702)
gray_color = (128, 128, 128)  # RGB values for gray color
new_image = Image.new("RGB", new_size, gray_color)

for image in tqdm(test_images):
    original_image_path = os.path.join(test_image_path, image)
    original_image = Image.open(original_image_path)

    # Calculate the position to paste the original image with padding
    x_offset = int((new_size[0] - original_image.size[0]) / 2)
    y_offset = int((new_size[1] - original_image.size[1]) / 2)

    # Paste the original image onto the new image with padding
    new_image.paste(original_image, (x_offset, y_offset))

#     plt.imshow(new_image)
    # Save the resized image with padding
    resized_image_path = os.path.join(new_path, image  )
    new_image.save(resized_image_path)
    # Optional: Display the resized image
#     new_image.show()


100%|██████████| 2/2 [00:00<00:00, 11.22it/s]


In [28]:
# train_meta = MetadataCatalog.get("CFH_test_padded_v2_rotated").set(box_mode=detectron2.structures.BoxMode(4))
# test_dict = DatasetCatalog.get("CFH_test_padded_v2_rotated")


test_image_path = find_latest_folder(r"C:\Users\AVLguest\Fiber Semantic Segmentation\input_padded_images")
test_images = [name for name in os.listdir(test_image_path) if '.png' in name]

combined_results_path_new = create_new_folder(combined_results_path)
print(combined_results_path_new)

output_path = os.path.basename(combined_results_path_new)
output_path = os.path.join(combined_results_path_new, output_path + '.xlsx')

data = []
for d in tqdm(test_images):
    test_path = os.path.join(test_image_path,d)
    
    im = cv2.imread(test_path)
    #im = cv2.imread(test_image_path)

    outputs_RFRCNN = predictor_RFRCNN(im)
    myoutput = outputs_RFRCNN['instances'].pred_boxes.tensor.cpu().detach().numpy()

    outputs_PRSS = predictor_PRSS(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
#     print(outputs_PRSS.keys())
    myseg = torch.argmax(outputs_PRSS['sem_seg'],0).cpu().detach().numpy()

    newseg=(~myseg.astype(bool)).astype(int) # reverse binary digits for mask (change to mapping for more than 2 classes)
    newseg = (newseg*255).astype(np.uint8)
    white_pixel_count = cv2.countNonZero(newseg)
    data.append([d, white_pixel_count])
        
#     print(np.unique(newseg))
    newseg = cv2.cvtColor(newseg,cv2.COLOR_GRAY2RGB)

    for box in myoutput:
        new_box = ((box[0],box[1]),(box[2],box[3]),(-box[4]))
        new_box = cv2.boxPoints(new_box)
        new_box = np.int0(new_box)
        cv2.drawContours(newseg,[new_box],0,(0,0,255),3)
    
    cv2.imwrite(os.path.join(combined_results_path_new,d), newseg)
dataframe = pd.DataFrame(data, columns=["Image Name", "White Pixel Count"])
# dataframe.to_excel(output_path, index = False)


C:\Users\AVLguest\Fiber Semantic Segmentation\Segmentation_output\2023-08-12_23-21-31


  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\AVLguest\.conda\envs\CFH\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
C:\Users\AVLguest\detectron2\projects\PointRend\point_rend\point_features.py:142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  point_coords[:, :, 1] = h_step / 2.0 + (point_indices // W).to(torch.float) * h_step
 50%|█████     | 1/2 [00:03<00:03,  3.11s/it]C:\Users\AVLguest\detectron2\projects\PointRend\